<a href="https://colab.research.google.com/github/mekintown/-data-science-and-data-engineering/blob/main/assignment5_1_6430324921.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import the libraries
import numpy as np
import pandas as pd

In [ ]:
# 1
df = pd.read_csv('https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week03_ML/mushroom2020_dataset.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           5824 non-null   int64  
 1   label                        5764 non-null   object 
 2   cap-shape                    5824 non-null   object 
 3   cap-surface                  5797 non-null   object 
 4   bruises                      5725 non-null   object 
 5   odor                         5725 non-null   object 
 6   gill-attachment              5725 non-null   object 
 7   gill-spacing                 5694 non-null   object 
 8   gill-size                    5703 non-null   object 
 9   stalk-shape                  5703 non-null   object 
 10  stalk-root                   5793 non-null   object 
 11  stalk-surface-above-ring     5793 non-null   object 
 12  stalk-surface-below-ring     5793 non-null   object 
 13  veil-type         

In [ ]:
df['gill-size'].isnull().sum()

121

In [ ]:
# 2
df.dropna(subset=['label'], inplace=True)

In [ ]:
# 3
columns_to_drop = [
    'id', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color-rate',
    'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',
    'stalk-color-above-ring-rate', 'stalk-color-below-ring-rate',
    'veil-color-rate', 'veil-type'
]

df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# 4
df.shape, df.isnull().sum()

((5764, 12),
 label                  0
 cap-shape              0
 cap-surface           27
 bruises               99
 odor                  99
 stalk-shape          121
 ring-number           62
 ring-type             62
 spore-print-color     56
 population            56
 habitat               31
 cap-color-rate        27
 dtype: int64)

In [ ]:
# 5
df.fillna(df.mean(), inplace=True)
df.fillna(df.mode().iloc[0], inplace=True)

df.isnull().sum()

<ipython-input-104-2231b88f7ea3>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


label                0
cap-shape            0
cap-surface          0
bruises              0
odor                 0
stalk-shape          0
ring-number          0
ring-type            0
spore-print-color    0
population           0
habitat              0
cap-color-rate       0
dtype: int64

In [ ]:
# 6
df['label'] = df['label'].map({'e': 1, 'p': 0})
df['label'].value_counts()

0    3660
1    2104
Name: label, dtype: int64

In [ ]:
# 7
nominal_columns = df.select_dtypes(include=["object"]).columns
dummy_df = pd.get_dummies(df[nominal_columns], drop_first=True)
df = pd.concat([df, dummy_df], axis=1)

df.drop(nominal_columns, axis=1, inplace=True)
df.to_csv('df_dummy.csv')
print(df.columns)
print(df)
print(df.shape)

Index(['label', 'cap-color-rate', 'cap-shape_c', 'cap-shape_f', 'cap-shape_k',
       'cap-shape_x', 'cap-surface_g', 'cap-surface_s', 'cap-surface_y',
       'bruises_t', 'odor_c', 'odor_f', 'odor_l', 'odor_m', 'odor_n', 'odor_p',
       'odor_s', 'odor_y', 'stalk-shape_t', 'ring-number_o', 'ring-number_t',
       'ring-type_f', 'ring-type_l', 'ring-type_n', 'ring-type_p',
       'spore-print-color_h', 'spore-print-color_k', 'spore-print-color_n',
       'spore-print-color_o', 'spore-print-color_r', 'spore-print-color_w',
       'spore-print-color_y', 'population_c', 'population_n', 'population_s',
       'population_v', 'population_y', 'habitat_g', 'habitat_l', 'habitat_m',
       'habitat_p', 'habitat_u', 'habitat_w'],
      dtype='object')
      label  cap-color-rate  cap-shape_c  cap-shape_f  cap-shape_k  \
0         0             1.0            0            0            0   
1         1             2.0            0            0            0   
2         1             3.0         

In [ ]:
from sklearn.model_selection import train_test_split
y = df.pop('label')
X = df

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=2020)
print(X_train.shape)

(4611, 42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 3, 6],
    'min_samples_leaf': [2, 5, 10],
    'n_estimators': [100, 200],
    'random_state': [2020]
}

rf_classifier = RandomForestClassifier()
grid_search = GridSearchCV(rf_classifier, param_grid=param_grid, cv=StratifiedKFold(n_splits=5))
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2, 'n_estimators': 100, 'random_state': 2020}


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

conf_matrix, class_report

(array([[731,   1],
        [  5, 416]]),
 '              precision    recall  f1-score   support\n\n           0       0.99      1.00      1.00       732\n           1       1.00      0.99      0.99       421\n\n    accuracy                           0.99      1153\n   macro avg       1.00      0.99      0.99      1153\nweighted avg       0.99      0.99      0.99      1153\n')

In [ ]:
len(X_train), len(X_test)

(4611, 1153)

In [ ]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 6,
 'min_samples_leaf': 2,
 'n_estimators': 100,
 'random_state': 2020}

In [ ]:
print(class_report)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       732
           1       1.00      0.99      0.99       421

    accuracy                           0.99      1153
   macro avg       1.00      0.99      0.99      1153
weighted avg       0.99      0.99      0.99      1153



In [ ]:
# -----------------------------------------
df1 = pd.read_csv('df_dummy.csv')
df2 = pd.read_csv('dummy_df.csv')

In [ ]:
df1_cols = df1.columns
df2_cols = df2.columns

In [ ]:
df1_cols

Index(['Unnamed: 0', 'label', 'cap-color-rate', 'cap-shape_c', 'cap-shape_f',
       'cap-shape_k', 'cap-shape_x', 'cap-surface_g', 'cap-surface_s',
       'cap-surface_y', 'bruises_t', 'odor_c', 'odor_f', 'odor_l', 'odor_m',
       'odor_n', 'odor_p', 'odor_s', 'odor_y', 'stalk-shape_t',
       'ring-number_o', 'ring-number_t', 'ring-type_f', 'ring-type_l',
       'ring-type_n', 'ring-type_p', 'spore-print-color_h',
       'spore-print-color_k', 'spore-print-color_n', 'spore-print-color_o',
       'spore-print-color_r', 'spore-print-color_w', 'spore-print-color_y',
       'population_c', 'population_n', 'population_s', 'population_v',
       'population_y', 'habitat_g', 'habitat_l', 'habitat_m', 'habitat_p',
       'habitat_u', 'habitat_w'],
      dtype='object')

In [ ]:
df2_cols

Index(['Unnamed: 0', 'cap-shape_c', 'cap-shape_f', 'cap-shape_k',
       'cap-shape_x', 'cap-surface_g', 'cap-surface_s', 'cap-surface_y',
       'bruises_t', 'odor_c', 'odor_f', 'odor_l', 'odor_m', 'odor_n', 'odor_p',
       'odor_s', 'odor_y', 'stalk-shape_t', 'ring-number_o', 'ring-number_t',
       'ring-type_f', 'ring-type_l', 'ring-type_n', 'ring-type_p',
       'spore-print-color_h', 'spore-print-color_k', 'spore-print-color_n',
       'spore-print-color_o', 'spore-print-color_r', 'spore-print-color_w',
       'spore-print-color_y', 'population_c', 'population_n', 'population_s',
       'population_v', 'population_y', 'habitat_g', 'habitat_l', 'habitat_m',
       'habitat_p', 'habitat_u', 'habitat_w', 'label', 'cap-color-rate'],
      dtype='object')

In [ ]:
df1 = df1.reindex(columns=df1_cols)
df2 = df2.reindex(columns=df1_cols)

In [ ]:
df1.columns

Index(['Unnamed: 0', 'label', 'cap-color-rate', 'cap-shape_c', 'cap-shape_f',
       'cap-shape_k', 'cap-shape_x', 'cap-surface_g', 'cap-surface_s',
       'cap-surface_y', 'bruises_t', 'odor_c', 'odor_f', 'odor_l', 'odor_m',
       'odor_n', 'odor_p', 'odor_s', 'odor_y', 'stalk-shape_t',
       'ring-number_o', 'ring-number_t', 'ring-type_f', 'ring-type_l',
       'ring-type_n', 'ring-type_p', 'spore-print-color_h',
       'spore-print-color_k', 'spore-print-color_n', 'spore-print-color_o',
       'spore-print-color_r', 'spore-print-color_w', 'spore-print-color_y',
       'population_c', 'population_n', 'population_s', 'population_v',
       'population_y', 'habitat_g', 'habitat_l', 'habitat_m', 'habitat_p',
       'habitat_u', 'habitat_w'],
      dtype='object')

In [ ]:
df2.columns

Index(['Unnamed: 0', 'label', 'cap-color-rate', 'cap-shape_c', 'cap-shape_f',
       'cap-shape_k', 'cap-shape_x', 'cap-surface_g', 'cap-surface_s',
       'cap-surface_y', 'bruises_t', 'odor_c', 'odor_f', 'odor_l', 'odor_m',
       'odor_n', 'odor_p', 'odor_s', 'odor_y', 'stalk-shape_t',
       'ring-number_o', 'ring-number_t', 'ring-type_f', 'ring-type_l',
       'ring-type_n', 'ring-type_p', 'spore-print-color_h',
       'spore-print-color_k', 'spore-print-color_n', 'spore-print-color_o',
       'spore-print-color_r', 'spore-print-color_w', 'spore-print-color_y',
       'population_c', 'population_n', 'population_s', 'population_v',
       'population_y', 'habitat_g', 'habitat_l', 'habitat_m', 'habitat_p',
       'habitat_u', 'habitat_w'],
      dtype='object')

In [ ]:
print(df1.equals(df2))

True
